In [1]:
import torch

class ModelExporter:
    """
    A utility class to export a PyTorch model to an ONNX file.
    """
    def __init__(self, model_name):
        """
        Initialize the exporter with a model name.
        Args:
            model_name (str): The name of the model. This will be used as the ONNX file name.
        """
        self.model_name = model_name

    def export_to_onnx(self, model, dummy_input):
        """
        Exports the given PyTorch model to an ONNX file.
        Args:
            model (torch.nn.Module): The PyTorch model to be exported.
            dummy_input (torch.Tensor): A dummy input tensor that matches the input shape of the model.
        """
        # Generate the ONNX file name using the model name
        onnx_file_name = f"model_onnx/{self.model_name}.onnx"

        # Export the model to ONNX format
        torch.onnx.export(
            model,
            dummy_input,
            onnx_file_name,           # Name of the output ONNX file
            export_params=True,       # Include the trained parameters in the exported file
            opset_version=11,         # Specify the ONNX opset version
            do_constant_folding=True, # Perform constant folding optimization
            input_names=["input"],    # Name of the input tensor
            output_names=["output"]   # Name of the output tensor
        )
        print(f"The ONNX file has been saved as '{onnx_file_name}'.")

In [2]:
import torch

class CustomReLU(torch.nn.Module):
    """
    Custom implementation of the ReLU activation function.
    """
    def __init__(self):
        super(CustomReLU, self).__init__()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Applies the ReLU function element-wise: max(0, x).
        Args:
            x (torch.Tensor): Input tensor
        Returns:
            torch.Tensor: Output tensor with ReALU applied
        """
        return torch.maximum(x, torch.zeros_like(x))

In [3]:
import torch

class ConvolutionalLayer:
    """
    A utility class to create a convolutional layer with a custom ReLU activation.
    """
    @staticmethod
    def create(in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int) -> torch.nn.Sequential:
        """
        Creates a convolutional layer with a custom ReLU activation.
        Args:
            in_channels (int): Number of input channels
            out_channels (int): Number of output channels
            kernel_size (int): Size of the convolution kernel
            stride (int): Stride of the convolution
            padding (int): Padding added to the input
        Returns:
            torch.nn.Sequential: A sequential layer containing Conv2d and CustomReLU
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
            CustomReLU()
        )

In [4]:
import torch

class FullyConnectedLayer:
    """
    A utility class to create Fully Connected (FC) layers with Xavier initialization.
    """
    @staticmethod
    def Dense(input_dim: int, output_dim: int) -> torch.nn.Linear:
        """
        Creates an FC layer and applies Xavier initialization.
        Args:
            input_dim (int): Number of input features
            output_dim (int): Number of output features
        Returns:
            torch.nn.Linear: Initialized Fully-Connected layer
        """
        layer = torch.nn.Linear(input_dim, output_dim, bias=True)
        torch.nn.init.xavier_uniform_(layer.weight)
        return layer

In [5]:
import torch

class SeparableConvolutionLayer:
    """
    Implementation of separable convolution layers with three variants:
    1. ReLU applied at the back.
    2. ReLU applied at the front.
    3. No ReLU applied.

    Separable convolutions split the depthwise and pointwise convolution operations
    to reduce the number of parameters and computational cost.
    """

    @staticmethod
    def ReLU_None(in_channels: int, out_channels: int, stride=1) -> torch.nn.Sequential:
        """
        Separable convolution without any ReLU activation.
        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            stride (int, optional): Stride for the depthwise convolution. Default is 1.
        Returns:
            torch.nn.Sequential: Sequential layer implementing separable convolution.
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels),  # Depthwise convolution
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),  # Pointwise convolution
        )
        
    @staticmethod
    def ReLU_Front(in_channels: int, out_channels: int, stride=1) -> torch.nn.Sequential:
        """
        Separable convolution with ReLU activation applied before the convolution.
        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            stride (int, optional): Stride for the depthwise convolution. Default is 1.
        Returns:
            torch.nn.Sequential: Sequential layer implementing separable convolution with ReLU at the front.
        """
        return torch.nn.Sequential(
            CustomReLU(),  # Activation function applied before convolution
            torch.nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels),  # Depthwise convolution
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),  # Pointwise convolution
        )

    @staticmethod
    def ReLU_Back(in_channels: int, out_channels: int, stride=1) -> torch.nn.Sequential:
        """
        Separable convolution with ReLU activation applied after the convolution.
        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            stride (int, optional): Stride for the depthwise convolution. Default is 1.
        Returns:
            torch.nn.Sequential: Sequential layer implementing separable convolution with ReLU at the back.
        """
        return torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels),  # Depthwise convolution
            torch.nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),  # Pointwise convolution
            CustomReLU()  # Activation function applied after convolution
        )

In [6]:
import torch

class CustomXception(torch.nn.Module):
    """
    Implementation of the Xception model.
    Input: Image tensor (batch_size, 3, 299, 299)
    Output: Class scores (batch_size, 1000)
    """
    def __init__(self, dropout_rate=0.5):
        """
        Args:
            dropout_rate (float): Dropout probability (default=0.5)
        """
        super(CustomXception, self).__init__()

        # Entry Flow
        self.layer1 = ConvolutionalLayer.create(3, 32, 3, 2, 1)
        self.layer2 = ConvolutionalLayer.create(32, 64, 3, 1, 1)

        self.layer3 = torch.nn.Sequential(
            SeparableConvolutionLayer.ReLU_None(64, 128),
            SeparableConvolutionLayer.ReLU_Front(128, 128),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer4 = torch.nn.Sequential(
            SeparableConvolutionLayer.ReLU_Front(128, 256),
            SeparableConvolutionLayer.ReLU_Front(256, 256),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer5 = torch.nn.Sequential(
            SeparableConvolutionLayer.ReLU_Front(256, 728),
            SeparableConvolutionLayer.ReLU_Front(728, 728),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        # Middle Flow
        self.layer6 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer7 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer8 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer9 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer10 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer11 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer12 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        self.layer13 = torch.nn.Sequential(
            *[SeparableConvolutionLayer.ReLU_Front(728, 728) for _ in range(3)]
        )

        # Exit Flow
        self.layer14 = torch.nn.Sequential(
            SeparableConvolutionLayer.ReLU_Front(728, 728),
            SeparableConvolutionLayer.ReLU_Front(728, 1024),
            torch.nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.layer15 = SeparableConvolutionLayer.ReLU_Back(1024, 1536)

        self.layer16 = SeparableConvolutionLayer.ReLU_Back(1536, 2048)

        self.layer17 = torch.nn.AvgPool2d(kernel_size=10, stride=1, padding=0)

        # Fully Connected layers and dropout
        self.layer_drop = torch.nn.Dropout(p=dropout_rate)

        self.layer18 = FullyConnectedLayer.Dense(2048, 1000)

        # Residual Convolutional Layer
        self.residual_1 = torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.residual_2 = torch.nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.residual_3 = torch.nn.Conv2d(256, 728, kernel_size=3, stride=2, padding=1)
        self.residual_4 = torch.nn.Conv2d(728, 1024, kernel_size=3, stride=2, padding=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the model.
        Args:
            x (torch.Tensor): Input image tensor (batch_size, 3, 299, 299)
        Returns:
            torch.Tensor: Class scores (batch_size, 1000)
        """
        x = self.layer1(x)
        x = self.layer2(x)
        x_residual = x
        
        x = self.layer3(x)
        x_residual = self.residual_1(x_residual)
        x += x_residual
        x_residual = x
        
        x = self.layer4(x)
        x_residual = self.residual_2(x_residual)
        x += x_residual
        x_residual = x

        x = self.layer5(x)
        x_residual = self.residual_3(x_residual)
        x += x_residual
        x_residual = x
        
        x = self.layer6(x)
        x += x_residual
        x_residual = x
        
        x = self.layer7(x)
        x += x_residual
        x_residual = x
        
        x = self.layer8(x)
        x += x_residual
        x_residual = x
        
        x = self.layer9(x)
        x += x_residual
        x_residual = x
        
        x = self.layer10(x)
        x += x_residual
        x_residual = x
        
        x = self.layer11(x)
        x += x_residual
        x_residual = x
        
        x = self.layer12(x)
        x += x_residual
        x_residual = x
        
        x = self.layer13(x)
        x += x_residual
        x_residual = x
        
        x = self.layer14(x)
        x_residual = self.residual_4(x_residual)
        x += x_residual
        
        x = self.layer15(x)
        x = self.layer16(x)
        x = self.layer17(x)
        x = self.layer_drop(x)
        x = x.view(x.size(0), -1)  # Flatten
        
        x = self.layer18(x)
        return x

In [7]:
from torchsummary import summary

model = CustomXception()

print(model.__class__.__name__)
summary(model, input_size=(3, 299, 299))

CustomXception
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 150, 150]             896
        CustomReLU-2         [-1, 32, 150, 150]               0
            Conv2d-3         [-1, 64, 150, 150]          18,496
        CustomReLU-4         [-1, 64, 150, 150]               0
            Conv2d-5         [-1, 64, 150, 150]             640
            Conv2d-6        [-1, 128, 150, 150]           8,320
        CustomReLU-7        [-1, 128, 150, 150]               0
            Conv2d-8        [-1, 128, 150, 150]           1,280
            Conv2d-9        [-1, 128, 150, 150]          16,512
        MaxPool2d-10          [-1, 128, 75, 75]               0
           Conv2d-11          [-1, 128, 75, 75]          73,856
       CustomReLU-12          [-1, 128, 75, 75]               0
           Conv2d-13          [-1, 128, 75, 75]           1,280
           Conv2d-14    